# Module version used

- torch 1.4.0
- numpy 1.18.1
- CPython 3.6.10
- IPython 7.12.0

- Runs on CPU or GPU (if available)

# Multilayer Perceptron

## Imports

In [6]:
import time
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Settings and Dataset

In [7]:
##########################
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64

# Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='/Users/kundan/MEGA/DatabaseLocal/', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='/Users/kundan/MEGA/DatabaseLocal/', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [8]:
##########################
### MODEL
##########################

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        # The following two lines are not necessary, 
        # but used here to demonstrate how to access the weights
        # and use a different weight initialization.
        # By default, PyTorch uses Xavier/Glorot initialization, which
        # should usually be preferred.
        self.linear_1.weight.detach().normal_(0.0, 0.1)
        self.linear_1.bias.detach().zero_()
        
        ### 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_2.weight.detach().normal_(0.0, 0.1)
        self.linear_2.bias.detach().zero_()
        
        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        self.linear_out.weight.detach().normal_(0.0, 0.1)
        self.linear_out.bias.detach().zero_()
        
    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.log_softmax(logits, dim=1)
        return logits, probas

    
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)

model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [9]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            logits, probas = net(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 2.4231
Epoch: 001/010 | Batch 050/938 | Cost: 0.7213
Epoch: 001/010 | Batch 100/938 | Cost: 0.3137
Epoch: 001/010 | Batch 150/938 | Cost: 0.4774
Epoch: 001/010 | Batch 200/938 | Cost: 0.3311
Epoch: 001/010 | Batch 250/938 | Cost: 0.3081
Epoch: 001/010 | Batch 300/938 | Cost: 0.3944
Epoch: 001/010 | Batch 350/938 | Cost: 0.2296
Epoch: 001/010 | Batch 400/938 | Cost: 0.0828
Epoch: 001/010 | Batch 450/938 | Cost: 0.2247
Epoch: 001/010 | Batch 500/938 | Cost: 0.2467
Epoch: 001/010 | Batch 550/938 | Cost: 0.1541
Epoch: 001/010 | Batch 600/938 | Cost: 0.2410
Epoch: 001/010 | Batch 650/938 | Cost: 0.4141
Epoch: 001/010 | Batch 700/938 | Cost: 0.2909
Epoch: 001/010 | Batch 750/938 | Cost: 0.3528
Epoch: 001/010 | Batch 800/938 | Cost: 0.1384
Epoch: 001/010 | Batch 850/938 | Cost: 0.0687
Epoch: 001/010 | Batch 900/938 | Cost: 0.2804
Epoch: 001/010 training accuracy: 94.88%
Time elapsed: 0.40 min
Epoch: 002/010 | Batch 000/938 | Cost: 0.3307
Epoch: 002/010 |

Epoch: 009/010 | Batch 750/938 | Cost: 0.0645
Epoch: 009/010 | Batch 800/938 | Cost: 0.0067
Epoch: 009/010 | Batch 850/938 | Cost: 0.1581
Epoch: 009/010 | Batch 900/938 | Cost: 0.0030
Epoch: 009/010 training accuracy: 99.33%
Time elapsed: 3.24 min
Epoch: 010/010 | Batch 000/938 | Cost: 0.0229
Epoch: 010/010 | Batch 050/938 | Cost: 0.0162
Epoch: 010/010 | Batch 100/938 | Cost: 0.0152
Epoch: 010/010 | Batch 150/938 | Cost: 0.0103
Epoch: 010/010 | Batch 200/938 | Cost: 0.0409
Epoch: 010/010 | Batch 250/938 | Cost: 0.0212
Epoch: 010/010 | Batch 300/938 | Cost: 0.0116
Epoch: 010/010 | Batch 350/938 | Cost: 0.0180
Epoch: 010/010 | Batch 400/938 | Cost: 0.0016
Epoch: 010/010 | Batch 450/938 | Cost: 0.0090
Epoch: 010/010 | Batch 500/938 | Cost: 0.0857
Epoch: 010/010 | Batch 550/938 | Cost: 0.0129
Epoch: 010/010 | Batch 600/938 | Cost: 0.0306
Epoch: 010/010 | Batch 650/938 | Cost: 0.0465
Epoch: 010/010 | Batch 700/938 | Cost: 0.0180
Epoch: 010/010 | Batch 750/938 | Cost: 0.0093
Epoch: 010/010 |

In [10]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.55%
